In [1]:
import numpy as np
from triqs.gf import *
from triqs.gf.meshes import MeshDLRImFreq, MeshDLRImTime
import numpy as np
import matplotlib.pylab as plt
import time


from triqs_tprf.lattice import iw_to_tau
from triqs_tprf.lattice import iw_to_tau_p
from triqs_tprf.lattice import iw_to_tau_p2
from triqs_tprf.lattice import tau_to_iw
from triqs_tprf.lattice import tau_to_iw_p
from triqs_tprf.lattice import inv

In [10]:
def get_cpu_info():
    cpu_info = {}
    with open('/proc/cpuinfo', 'r') as cpuinfo:
        for line in cpuinfo:
            if line.strip():
                key, value = line.strip().split(':')
                cpu_info[key.strip()] = value.strip()
    return cpu_info

def get_threads_per_core():
    cpu_info = get_cpu_info()
    siblings = int(cpu_info.get('siblings', 1))
    cores = int(cpu_info.get('cpu cores', 1))
    threads_per_core = siblings // cores
    return threads_per_core


def generate_G(tij, mesh, spin_names = ['up', 'dn']):
    g_inv = Gf(mesh = mesh, target_shape = np.shape(tij))
    g_inv << iOmega_n - tij.transpose()
    g = inv(g_inv, 8)
    return BlockGf(block_list = [g] * 2, name_list = spin_names, make_copies = False)


orbitals = 200
t = 1.0
tij = np.zeros([orbitals] * 2)
for i in range(orbitals - 1):
    tij[i, i + 1] = -t
    tij[i + 1, i] = -t


iw_mesh = MeshDLRImFreq(beta = 100, statistic = 'Fermion', w_max = 20.0, eps = 1e-12, symmetrize = True)
G_iw = generate_G(tij, iw_mesh)


In [3]:
# start = time.perf_counter()
# G_tau1 = iw_to_tau(G_iw)
# print(time.perf_counter() - start)

3.1455929369999467


In [11]:
start = time.perf_counter()
G_tau2 = iw_to_tau_p(G_iw, 8)
print(time.perf_counter() - start)

0.26807332000021233


In [12]:
start = time.perf_counter()
G_tau3 = iw_to_tau_p2(G_iw, 8)
print(time.perf_counter() - start)

0.3644470070000807


In [6]:
# np.testing.assert_allclose(G_tau1['up'].data, G_tau2['up'].data)
# np.testing.assert_allclose(G_tau1['dn'].data, G_tau2['dn'].data)
# np.testing.assert_allclose(G_tau1['dn'].data, G_tau3['dn'].data)